In [ ]:
import math
import numpy as np

class projectilesolver(object):

    def __init__(self, lat1, lon1, alt1, lat2, lon2, alt2):
        self.lat1 = lat1
        self.lon1 = lon1
        self.alt1 = alt1  # Altitude in kilometers (e.g., 0.1 = 100m)
        self.lat2 = lat2
        self.lon2 = lon2
        self.alt2 = alt2

    def compute_orientation(self):
        # Conversion: 1 degree ≈ 111 km
        scale = 111

        # Compute horizontal dx, dy (on Earth's surface)
        mean_lat = math.radians((self.lat1 + self.lat2) / 2.0)
        dx = (self.lon2 - self.lon1) * scale * math.cos(mean_lat)
        dy = (self.lat2 - self.lat1) * scale
        dz = self.alt2 - self.alt1  # vertical difference in km

        # Azimuth angle (in horizontal plane, from x-axis/east)
        azimuth_rad = math.atan2(dy, dx)
        azimuth_deg_east = (math.degrees(azimuth_rad)+360) % 360

        #convert to azimuth from North
        azimuth_from_north=(90-azimuth_deg_east)% 360

        # Horizontal ground distance (hypotenuse in 2D plane)
        horizontal_distance = math.sqrt(dx**2 + dy**2)

        # Elevation angle (from ground up/down)
        elevation_rad = math.atan2(dz, horizontal_distance)
        elevation_deg = math.degrees(elevation_rad)

        return azimuth_from_north, elevation_deg, horizontal_distance, dz



    #############  Range #############
    def haversine_distance(self):
        # Radius of Earth in kilometers. Use 3956 for miles
        R = 6371.0

        # Convert degrees to radians
        lat1_rad = math.radians(self.lat1)
        lon1_rad = math.radians(self.lon1)
        lat2_rad = math.radians(self.lat2)
        lon2_rad = math.radians(self.lon2)

        # Differences
        dlat = lat2_rad - lat1_rad
        dlon = lon2_rad - lon1_rad

        # Haversine formula
        a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

        distance_km = R * c  # in kilometers
        print(f'distance in KM range :: {distance_km}')
        return distance_km

    ############ calculated Range ##############
    def shooting(self, initial_velocity, angle_of_projection):
        u = initial_velocity  # (km/s)
        g = 9.8 * (pow(10, -3))
        theta = math.radians(angle_of_projection)  # convert degrees to radians if needed
        result = math.sin(2 * theta)

        range1 = (pow(u, 2)) * result / g
        
        actual_distance = self.haversine_distance()
        print(f'your calculated RANGE according to initial velocity and angle :: {range1}')
        if abs(range1 - actual_distance) < 1e-1:
            return ' #####  TARGIT HIT  #### '
        else:
            miss = actual_distance - range1
            if miss > 0:
                print(' ####   target is ahead of u   ####')
            else:
                print('####  u crossed target  ####')
            return f'You missed target by a distance :: {abs(miss)} Km'

    ################    Time of FLight   #################

    def time_of_flight(self, initial_velocity, angle_of_projection):
        u = initial_velocity  # (km/s)
        g = 9.8 * (pow(10, -3))  # gravitational constant
        theta = math.radians(angle_of_projection)  # convert degrees to radians if needed
        result = math.sin(theta)

        time = (2 * u * result) / g
        print(f'Time of flight in seconds :: {time}')
        return time
    
    ################  max height        ##############
    def max_height(self, initial_velocity, angle_of_projection):
        u = initial_velocity  # (km/s)
        g = 9.8 * (pow(10, -3))  # gravitational constant
        theta = math.radians(angle_of_projection)  # convert degrees to radians if needed
        result = math.sin(theta)

        max_height = (pow(u, 2)) * (pow(result, 2)) / (2 * g)
        print(f'max_height Km :: {max_height}')
        return max_height
    

    ###########  Optimal angle ################
    # Optimal angle of projection as we know our enemy location
    def optimal_angle(self, initial_velocity):
        u = initial_velocity
        R = self.haversine_distance()
        g = 9.8 * 1e-3

        value = (R * g) / (u ** 2)
        if value >= -1 and value <=1:
            angle_rad = math.asin(value) / 2
            angle_deg = math.degrees(angle_rad)
            print(f'Optimal projection angle {angle_deg}')
        else:
            print(f'with that initial velocity optimal angle not possible ')
          # clamp for safe asin()

        
        # return angle_deg
    
    ############  trajectory equation ##############
    def trajectory_equation(self, initial_velocity, angle_of_projection):
        """
        Returns the trajectory equation coefficients in the form:
        y = -a*x^2 + b*x (i.e., y = ax^2 + bx + c, with c = 0)
        where:
        - a = g / (2 * u^2 * cos^2(theta))
        - b = tan(theta)
        """
        u = initial_velocity
        g = 9.8 * 1e-3  # km/s^2
        theta = math.radians(angle_of_projection)

        cos_theta = math.cos(theta)
        if cos_theta == 0:
            raise ValueError("cos(theta) is zero, invalid angle")

        a = g / (2 * u ** 2 * cos_theta ** 2)
        b = math.tan(theta)

        print(f"Parabolic Trajectory Equation: y = -({a:.4f})x² + ({b:.4f})x")
        return -a, b  # 'a' is negative in projectile motion



# input parameters :: initial_velocity
#                     angle of projection 
#                     enemy and our location
lat1, lon1, alt1 = 17.5959743, 78.1244778,0.0        # your position :sangareddy
lat2, lon2, alt2 = 17.5847311, 78.121341, 0.00       #  enemyposition:  hyderabad
initial_velocity = 0.4
angle_of_projection = 18.4997893488487

# object by passing coordinates
solver = projectilesolver( lat1, lon1, alt1, lat2, lon2, alt2)

# Now call the methods
solver =projectilesolver(lat1, lon1, alt1, lat2, lon2, alt2)
azimuth, elevation, ground_range, height_diff = solver.compute_orientation()

print(f"Azimuth (from East): {azimuth:}°")
print(f"Elevation (upward): {elevation:}°")
print(f"Horizontal range: {ground_range:} km")
print(f"Height difference: {height_diff * 1000:.1f} m")


solver.haversine_distance()
print(solver.shooting(initial_velocity, angle_of_projection))
solver.time_of_flight(initial_velocity, angle_of_projection)
solver.max_height(initial_velocity, angle_of_projection)
solver.optimal_angle(initial_velocity)
solver. trajectory_equation( initial_velocity, angle_of_projection)
